In [1]:
!which python

/home/celidos/anaconda3/envs/venv_tensorflowgpu/bin/python


In [2]:
!pip install pymongo

In [301]:
import pymongo
import pandas as pd
from collections import defaultdict

In [4]:
client = pymongo.MongoClient("176.99.11.79:27017", username="root", password="example")

db = client.declarations

In [5]:
db.persons.count()

/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


132784

In [19]:
db.persons.find_one()

{'_id': 8,
 'declarations': [{'main': {'person': {'id': 8,
     'name': 'Зюганов Геннадий Андреевич',
     'family_name': 'Зюганов',
     'given_name': 'Геннадий',
     'patronymic_name': 'Андреевич',
     'gender': 'M'},
    'office': {'id': 14,
     'name': 'Государственная Дума',
     'url': 'http://www.duma.gov.ru/',
     'type': {'id': 7, 'name': 'Федеральный, без региональной структуры'},
     'region': None},
    'party': {'id': 6, 'name': 'КПРФ'},
    'year': 2010,
    'document_type': {'id': 2, 'name': 'Предвыборная декларация'}},
   'incomes': [{'size': 2126487.0,
     'relative': None,
     'comment': 'пенсия,  Аппарат Государственной Думы Федерального собрания Российской Федерации, доходы от вкладов'}],
   'real_estates': [{'name': '',
     'square': 167.4,
     'country': 'Россия',
     'region': {'id': 63, 'name': 'Москва'},
     'comment': '',
     'type': {'id': 4, 'name': 'Квартира'},
     'own_type': {'id': 22, 'name': 'Совместная собственность'},
     'share': 0.25,


#### все возможные места работы

In [78]:
pipeline = [ 
    {"$unwind": "$declarations"},
    #{"$project": {"docsTop10": { "$slice": [ "$declarations", 10 ] }}}
    
#     {"$group" : 
#         {"_id": "null",
#          "selectedoffices": { "$push": "$declarations.main.office" }      
#     }},
#     {"$project": {
#         "selectedoffices": { "$reduce": {
#             "input": "$selectedoffices",
#             "initialValue": [],
#             "in": {"$setUnion" : ["$$value", "$$this"]}
#         }}
#     }}
    {"$group": {"_id": "null", "uniqueValues": {"$addToSet": "$declarations.main.office"}}},
]

In [93]:
office_places = list(db.persons.aggregate(pipeline))[0]['uniqueValues']

In [87]:
len(office_places)

3246

In [98]:
office_places[:5]

[{'id': 4364,
  'name': 'Главное управление МВД России Кемеровская область',
  'url': 'https://42.mvd.ru/',
  'type': {'id': 18, 'name': 'Региональный, без структуры'},
  'region': {'id': 50, 'name': 'Кемеровская область'}},
 {'id': 687,
  'name': 'Арбитражный суд Приморского края',
  'url': 'http://www.primkray.arbitr.ru/',
  'type': {'id': 18, 'name': 'Региональный, без структуры'},
  'region': {'id': 74, 'name': 'Приморский край'}},
 {'id': 381,
  'name': 'Избирательная комиссия Чеченской республики',
  'url': 'http://www.chechen.izbirkom.ru/',
  'type': {'id': 7, 'name': 'Федеральный, без региональной структуры'},
  'region': {'id': 90, 'name': 'Чеченская республика'}},
 {'id': 4566,
  'name': 'Управление Роскомнадзора Ярославская область',
  'url': 'http://76.rkn.gov.ru/',
  'type': {'id': 18, 'name': 'Региональный, без структуры'},
  'region': {'id': 107, 'name': 'Ярославская область'}},
 {'id': 4892,
  'name': 'Управление Федеральной службы судебных приставов Вологодская область

In [96]:
different_office_places = set()
for item in office_places:
    if 'region' in item:
        if item['region'] is not None:
            different_office_places.add((item['region']['id'], item['region']['name']))

In [213]:
different_office_places = list(different_office_places)

In [214]:
different_office_places

[(81, 'Ставропольский край'),
 (21, 'Республика Хакасия'),
 (97, 'Калужская область'),
 (27, 'Краснодарский край'),
 (63, 'Москва'),
 (88, 'Ульяновская область'),
 (8, 'Республика Ингушетия'),
 (75, 'Псковская область'),
 (17, 'Республика Северная Осетия — Алания'),
 (3, 'Республика Адыгея'),
 (90, 'Чеченская республика'),
 (36, 'Владимирская область'),
 (59, 'Республика Дагестан'),
 (51, 'Кировская область'),
 (18, 'Республика Татарстан'),
 (96, 'Белгородская область'),
 (20, 'Удмуртская республика'),
 (67, 'Новгородская область'),
 (94, 'Сахалинская область'),
 (55, 'Курская область'),
 (61, 'Республика Марий Эл'),
 (72, 'Пензенская область'),
 (77, 'Самарская область'),
 (56, 'Ленинградская область'),
 (107, 'Ярославская область'),
 (65, 'Мурманская область'),
 (68, 'Новосибирская область'),
 (62, 'Республика Мордовия'),
 (64, 'Московская область'),
 (5, 'Республика Бурятия'),
 (91, 'Чувашская республика - Чувашия'),
 (66, 'Нижегородская область'),
 (57, 'Липецкая область'),
 (1, 'С

In [215]:
place2id = {x[1]: x[0] for x in different_office_places}
id2place = {x[0]: x[1] for x in different_office_places}

In [88]:
office_places[:5]

[{'id': 5,
  'name': 'Совет Федерации',
  'url': 'http://council.gov.ru/',
  'type': {'id': 7, 'name': 'Федеральный, без региональной структуры'},
  'region': None},
 {'id': 4200,
  'name': 'Федеральное агентство по делам национальностей',
  'url': 'http://fadn.gov.ru/',
  'type': {'id': 7, 'name': 'Федеральный, без региональной структуры'},
  'region': None},
 {'id': 391,
  'name': 'Избирательная комиссия Хабаровского края',
  'url': 'http://www.khabarovsk.izbirkom.ru/',
  'type': {'id': 7, 'name': 'Федеральный, без региональной структуры'},
  'region': {'id': 32, 'name': 'Хабаровский край'}},
 {'id': 5014,
  'name': 'Территориальный орган Федеральной службы государственной статистики Республика Северная Осетия — Алания  (присоединен к Росстат по СКФО)',
  'url': 'http://osetstat.gks.ru/',
  'type': {'id': 18, 'name': 'Региональный, без структуры'},
  'region': {'id': 17, 'name': 'Республика Северная Осетия — Алания'}},
 {'id': 4014,
  'name': 'Зарубежнефть',
  'url': 'http://www.nest

# ---------------------- Средняя зарплата женская и мужская ----------------------

In [400]:
def get_mean_gender(gender='M', minimal_year=2012):
    pipeline = [
        {"$unwind": "$declarations"},
        #{"$match": {"declarations.main.office.region": {"$exists": "true", "$ne": None} }},
        {"$match": {"declarations.main.person.gender": {"$eq": gender}}},
        {"$match": {"declarations.main.year": {"$gte": minimal_year}}},
        {"$unwind": "$declarations.incomes"},
        {"$match": {"eclarations.incomes.relative": {"$eq": None}}},
        {"$group": {"_id": {"region": "$declarations.main.office.region.id",
                           "person": "$declarations.main.person.id"}, 
                    "sumincomes" : {"$avg" : "$declarations.incomes.size"}}},
    ]

    out = db.persons.aggregate(pipeline)
    out = list(out)
#     print(out)
    return out

In [401]:
dataM = get_mean_gender('M')

In [402]:
dataF = get_mean_gender('F')

In [403]:
it = 0
man_cnt = 0
man_mean_salad = 0
for record in dataM:
    man_cnt += 1
    man_mean_salad += record['sumincomes']

In [404]:
print(man_cnt, man_mean_salad / man_cnt)

81359 2044225.0797556022


In [405]:
it = 0
fem_cnt = 0
fem_mean_salad = 0
for record in dataF:
    fem_cnt += 1
    fem_mean_salad += record['sumincomes']

In [406]:
print(fem_cnt, fem_mean_salad / man_cnt)

36861 675300.8411859042


In [384]:
len(dataM)

81359

In [407]:
(fem_mean_salad + man_mean_salad) / (man_cnt + fem_cnt)

1871577.64677618

In [408]:
fem_cnt + man_cnt

118220

## Демографический по регионам

In [253]:
def get_by_gender(gender='M', minimal_year=2012):
    pipeline = [
        {"$unwind": "$declarations"},
        {"$match": {"declarations.main.office.region": {"$exists": "true", "$ne": None} }},
        {"$match": {"declarations.main.person.gender": {"$eq": gender}}},
        {"$match": {"declarations.main.year": {"$gte": minimal_year}}},
        {"$group": {"_id": "$declarations.main.office.region", "allincomes" : {"$push" : {"incomes": "$declarations.incomes"}}}},
    ]

    out = db.persons.aggregate(pipeline)
    out = list(out)
#     print(out)
    return out

In [254]:
out_man = get_by_gender(gender='M')
out_fem = get_by_gender(gender='F')

In [255]:
def extract_list(data):
    result = {}
    for item in data:
#     print(item)
        region_sum = 0
        region_total = 0
        for family in item['allincomes']:
            flag = 0
            for income in family['incomes']:
                if income['relative'] is not None:
                    flag = 1
                    region_sum += income['size']
            region_total += flag
        if region_total > 0: 
            result[item['_id']['id']] = (region_sum / region_total, region_total)
        else:
            result[item['_id']['id']] = (None, region_total)
    return result

In [256]:
list_man = extract_list(out_man)
list_fem = extract_list(out_fem)

In [257]:
list_man

{43: (2532386.1392048206, 415),
 101: (505498.85988262895, 426),
 80: (1340613.0604992318, 1302),
 37: (1173597.9525990502, 631),
 75: (1444850.4108243727, 279),
 72: (1587285.6455393585, 343),
 28: (971625.6787283961, 810),
 74: (1323291.0092792793, 555),
 93: (2405163.099871795, 156),
 103: (918045.3687087094, 333),
 46: (767366.4394647641, 1788),
 5: (1246837.2152663944, 488),
 8: (282252.5019815669, 217),
 47: (531289.849617225, 209),
 55: (1023127.6091277266, 321),
 29: (1262882.8930272954, 806),
 99: (625046.2346078436, 306),
 94: (1108453.521617021, 235),
 67: (830771.4876288656, 291),
 85: (859807.9093385214, 257),
 107: (965707.6578436653, 371),
 50: (635391.7285306113, 1225),
 76: (1890329.2825158252, 632),
 36: (1164225.653029089, 479),
 27: (1245080.933366763, 698),
 42: (621752.6222477875, 565),
 20: (1144542.062331378, 682),
 86: (1481560.572632979, 376),
 109: (1159445.9929213482, 89),
 59: (555485.7078640778, 515),
 44: (767783.050349463, 372),
 12: (1345015.0545977014,

# супруги

In [264]:
def get_all_decls():
    pipeline = [
        {"$unwind": "$declarations"},
        {"$match": {"declarations.main.office.region": {"$exists": "true", "$ne": None} }},
        #{"$match": {"declarations.main.person.gender": {"$eq": gender}}},
        #{"$match": {"declarations.main.year": {"$eq": year}}},
        {"$unwind": "$declarations.incomes"},
        {"$match": {"declarations.incomes.relative": {"$exists": "true", "$ne": None} }},
        {"$group": {"_id": "$declarations.main.office.region", "allincomes" : {"$push" : {"incomes": "$declarations.incomes"}}}},
    ]

    out = db.persons.aggregate(pipeline)
    out = list(out)
#     print(out)
    return out

In [265]:
siblings_data = get_all_decls()

In [275]:
siblings = {}
for region in siblings_data:
    income_sum = 0
    income_total = 0
    for people in region['allincomes']:
        income_sum += people['incomes']['size']
        income_total += 1
        
    if income_total > 0:
        siblings[region['_id']['id']] = income_sum / income_total
    else:
        siblings[region['_id']['id']] = None

In [276]:
siblings

{58: 944442.8064874548,
 63: 1359608.5702616677,
 66: 1058005.9021378518,
 84: 4411190.380958191,
 91: 627852.503799682,
 4: 915923.0340692241,
 98: 605394.4933647054,
 44: 767551.3176072233,
 12: 1214326.111960352,
 95: 1910395.9667910459,
 61: 655857.8458959531,
 65: 982154.7344208038,
 68: 1182804.311935678,
 100: 658097.4649295777,
 20: 1003717.9622260274,
 86: 1354305.0424593135,
 109: 968526.4901612905,
 59: 447441.66135766444,
 67: 684688.9911007462,
 85: 847964.0196675896,
 107: 834435.6845979381,
 42: 587047.2051522841,
 76: 1651767.8508750016,
 36: 1054991.9063933725,
 27: 1144179.889473685,
 50: 608834.2250241796,
 29: 1096969.3593829405,
 99: 705542.1215690868,
 94: 1113815.3670422544,
 103: 791666.1757362643,
 46: 658633.3174365113,
 5: 1055549.9677583477,
 8: 201710.77560679603,
 47: 495004.95520599256,
 55: 947912.7149608358,
 75: 1444589.1085752682,
 72: 1317424.171696428,
 28: 963710.2538909429,
 93: 1881997.0262389376,
 74: 2567325.4254106916,
 43: 2121368.7083918676,

# накопления

In [297]:
def get_all_savings():
    pipeline = [
        {"$unwind": "$declarations"},
        {"$match": {"declarations.main.office.region": {"$exists": "true", "$ne": None} }},

        {"$match": {"declarations.savings": {"$exists": "true", "$ne": None} }},
        {"$group": {"_id": {"region": "$declarations.main.office.region", "person": "$declarations.main.person.id"}, 
                    "allincomes" : {"$push" : {"savings": "$declarations.savings"}}}},
    ]

    out = db.persons.aggregate(pipeline)
    out = list(out)
#     print(out)
    return out

In [310]:
savings_data = get_all_savings()

In [311]:
def parse_saving(el):
    el = str(el)
    el = el.split('руб', 1)[0].split(',', 1)[0]
    el = el.replace(" ", "")
    if el:
        return int(el)
    return 0

In [312]:
savings_total = defaultdict(int)
savings_count = defaultdict(int)
for record in savings_data:
    cur_sum = 0
    
    reg_id = record['_id']['region']['id']
    flag = 0
    for saving in record['allincomes']:
#         print(saving)
        for el in saving['savings']:
            flag = 1
#             print(el)
            cur_sum += parse_saving(el)
    savings_total[reg_id] += cur_sum
    savings_count[reg_id] += flag

In [316]:
savings = {}
for key in savings_total.keys():
    if savings_total[key]:
        savings[key] = savings_total[key] / savings_count[key]
    else:
        savings[key] = None

In [317]:
savings

{24: None,
 85: None,
 4: None,
 84: None,
 37: None,
 68: None,
 35: None,
 1: 184073.8,
 46: None,
 57: None,
 63: 1696206.2727272727,
 48: 56184215.0,
 72: None,
 69: None,
 36: 1997065.3333333333,
 62: None,
 42: None,
 51: None,
 80: None,
 13: None,
 96: None,
 41: None,
 74: None,
 99: None,
 9: None,
 66: None,
 105: None,
 53: 4570899.0,
 81: None,
 20: None,
 98: None,
 56: None,
 43: None,
 65: None,
 29: None,
 59: None,
 94: 1790589.0,
 18: None,
 28: None,
 100: None,
 67: None,
 64: None,
 50: None,
 5: 3307389.2352941176,
 82: 127784.0,
 77: 20470347.0,
 79: None,
 32: None,
 104: None,
 101: None,
 88: None,
 93: 14724978.0,
 91: 2031993.0,
 11: None,
 103: None,
 106: None,
 21: None,
 27: None,
 76: None,
 86: None,
 102: None,
 108: None,
 95: 318129.0,
 89: None,
 45: 1814035.0,
 38: None,
 75: None,
 61: None,
 47: None,
 55: None,
 6: None,
 97: None,
 92: None,
 33: 29502675.0,
 70: None,
 107: None,
 12: None,
 90: None,
 8: 36478.0,
 17: None,
 110: None,
 3: 

# индекс роста

http://old.fa.ru/dep/press/about-us/Documents/2017_07.pdf

In [347]:
growdata = pd.read_csv('./growtable.csv', sep='\t')
growing = {}

for index, row in growdata.iterrows():
    growing[row['id']] = float(row['value'].replace(',', '.'))

In [348]:
growing

{27: 10.7,
 33: 8.9,
 92: 8.3,
 32: 8.0,
 74: 7.4,
 100: 7.3,
 56: 7.1,
 77: 6.4,
 57: 5.7,
 64: 5.7,
 97: 5.6,
 67: 5.3,
 36: 5.2,
 28: 4.6,
 84: 4.5,
 44: 4.5,
 101: 4.5,
 103: 4.5,
 105: 4.5,
 21: 4.4,
 50: 4.4,
 45: 4.1,
 96: 4.0,
 5: 4.0,
 35: 3.9,
 55: 3.8,
 4: 3.8,
 80: 3.7,
 63: 3.7,
 88: 3.7,
 68: 3.6,
 107: 3.5,
 69: 3.5,
 20: 3.4,
 46: 3.4,
 81: 3.3,
 24: 3.3,
 51: 3.3,
 79: 3.3,
 66: 3.2,
 70: 3.2,
 99: 3.2,
 42: 3.2,
 59: 3.1,
 18: 3.0,
 82: 2.9,
 29: 2.8,
 41: 2.7,
 98: 2.6,
 62: 2.6,
 61: 2.6,
 72: 2.4,
 9: 2.3,
 91: 2.3,
 75: 2.3,
 106: 2.3,
 89: 2.2,
 1: 2.1,
 86: 2.0,
 12: 1.9,
 90: 1.9,
 104: 1.7,
 109: 1.6,
 17: 1.5,
 37: 1.4,
 108: 1.1,
 53: 1.0,
 43: 0.8,
 38: 0.0,
 76: -0.2,
 13: -0.6,
 65: -1.0}

# collect all data

In [350]:
T = {}
for key, value in list_man.items():
    T[key] = {'man_mean': value[0], 'fem_mean': None, 'region_name': id2place[key]}
for key, value in list_fem.items():
    T[key]['fem_mean'] = value[0]
for key, value in siblings.items():
    T[key]['siblings_mean_income'] = value
for key, value in savings.items():
    T[key]['savings_mean'] = value
for key, value in growing.items():
    T[key]['growing_index'] = value

In [351]:
T

{43: {'man_mean': 2532386.1392048206,
  'fem_mean': 878433.8131506853,
  'region_name': 'Воронежская область',
  'siblings_mean_income': 2121368.7083918676,
  'savings_mean': None,
  'growing_index': 0.8},
 101: {'man_mean': 505498.85988262895,
  'fem_mean': 722144.2678378376,
  'region_name': 'Тверская область',
  'siblings_mean_income': 886915.0074319732,
  'savings_mean': None,
  'growing_index': 4.5},
 80: {'man_mean': 1340613.0604992318,
  'fem_mean': 1113433.1748704666,
  'region_name': 'Свердловская область',
  'siblings_mean_income': 1162254.1584916462,
  'savings_mean': None,
  'growing_index': 3.7},
 37: {'man_mean': 1173597.9525990502,
  'fem_mean': 1039881.1486086956,
  'region_name': 'Архангельская область',
  'siblings_mean_income': 1065313.6749280582,
  'savings_mean': None,
  'growing_index': 1.4},
 75: {'man_mean': 1444850.4108243727,
  'fem_mean': 919734.2317460316,
  'region_name': 'Псковская область',
  'siblings_mean_income': 1444589.1085752682,
  'savings_mean': N

In [352]:
gks = pd.read_csv('./gks_new.csv', sep='\t')

In [353]:
for index, row in gks.iterrows():
    dd = row['id']
    if dd in T:
        T[dd]['salary_mean'] = row['mean']

In [354]:
T

{43: {'man_mean': 2532386.1392048206,
  'fem_mean': 878433.8131506853,
  'region_name': 'Воронежская область',
  'siblings_mean_income': 2121368.7083918676,
  'savings_mean': None,
  'growing_index': 0.8,
  'salary_mean': 31207},
 101: {'man_mean': 505498.85988262895,
  'fem_mean': 722144.2678378376,
  'region_name': 'Тверская область',
  'siblings_mean_income': 886915.0074319732,
  'savings_mean': None,
  'growing_index': 4.5,
  'salary_mean': 31049},
 80: {'man_mean': 1340613.0604992318,
  'fem_mean': 1113433.1748704666,
  'region_name': 'Свердловская область',
  'siblings_mean_income': 1162254.1584916462,
  'savings_mean': None,
  'growing_index': 3.7,
  'salary_mean': 38052},
 37: {'man_mean': 1173597.9525990502,
  'fem_mean': 1039881.1486086956,
  'region_name': 'Архангельская область',
  'siblings_mean_income': 1065313.6749280582,
  'savings_mean': None,
  'growing_index': 1.4,
  'salary_mean': 48307},
 75: {'man_mean': 1444850.4108243727,
  'fem_mean': 919734.2317460316,
  'regi

In [356]:
pd.DataFrame(T).transpose().to_csv('regions_from_2012_siblings_genders_savings_growing.csv')

# ------------------------------------------

In [ ]:
{'_id': 8,
 'declarations': [{'main': {'person': {'id': 8,
     'name': 'Зюганов Геннадий Андреевич',
     'family_name': 'Зюганов',
     'given_name': 'Геннадий',
     'patronymic_name': 'Андреевич',
     'gender': 'M'},
    'office': {'id': 14,
     'name': 'Государственная Дума',
     'url': 'http://www.duma.gov.ru/',
     'type': {'id': 7, 'name': 'Федеральный, без региональной структуры'},
     'region': None},
    'party': {'id': 6, 'name': 'КПРФ'},
    'year': 2010,
    'document_type': {'id': 2, 'name': 'Предвыборная декларация'}},
   'incomes': [{'size': 2126487.0,
     'relative': None,
     'comment': 'пенсия,  Аппарат Государственной Думы Федерального собрания Российской Федерации, доходы от вкладов'}],
   'real_estates': [{'name': '',
     'square': 167.4,
     'country': 'Россия',
     'region': {'id': 63, 'name': 'Москва'},
     'comment': '',
     'type': {'id': 4, 'name': 'Квартира'},
     'own_type': {'id': 22, 'name': 'Совместная собственность'},
     'share': 0.25,
     'relative': None}],
   'vehicles': [],
   'savings': ['2 652 390,0 руб. (ВТБ 24 (ЗАО))',
    '474 061,0 руб. (ОАО "Сбербанк России")'],
   'stocks': [],
   'bonds': [],
   'spendings': []},
  {'main': {'person': {'id': 8,
     'name': 'Зюганов Геннадий Андреевич',
     'family_name': 'Зюганов',
     'given_name': 'Геннадий',
     'patronymic_name': 'Андреевич',
     'gender': 'M'},
    'office': {'id': 14,
     'name': 'Государственная Дума',
     'url': 'http://www.duma.gov.ru/',
     'type': {'id': 7, 'name': 'Федеральный, без региональной структуры'},
     'region': None},
    'party': {'id': 6, 'name': 'КПРФ'},
    'year': 2006,
    'document_type': {'id': 2, 'name': 'Предвыборная декларация'}},
   'incomes': [{'size': 1257784.0,
     'relative': None,
     'comment': 'Аппарат ГД ФС РФ, пенсия, доходы от вкладов'}],
   'real_estates': [{'name': '',
     'square': 167.4,
     'country': 'Россия',
     'region': {'id': 63, 'name': 'Москва'},
     'comment': '',
     'type': {'id': 4, 'name': 'Квартира'},
     'own_type': {'id': 20, 'name': 'Индивидуальная'},
     'share': 0.25,
     'relative': None}],
   'vehicles': [],
   'savings': ['141 740,91 руб. (ОАО "Сбербанк России")'],
   'stocks': [],
   'bonds': [],
   'spendings': []},
  {'main': {'person': {'id': 8,
     'name': 'Зюганов Геннадий Андреевич',
     'family_name': 'Зюганов',
     'given_name': 'Геннадий',
     'patronymic_name': 'Андреевич',
     'gender': 'M'},
    'office': {'id': 14,
     'name': 'Государственная Дума',
     'url': 'http://www.duma.gov.ru/',
     'type': {'id': 7, 'name': 'Федеральный, без региональной структуры'},
     'region': None},
    'party': {'id': 6, 'name': 'КПРФ'},
    'year': 2010,
    'document_type': {'id': 1, 'name': 'Антикоррупционная декларация'}},
   'incomes': [{'size': 2127686.49, 'relative': None, 'comment': ''},
    {'size': 168601.94,
     'relative': {'id': 2, 'name': 'Супруг(а)'},
     'comment': ''}],
   'real_estates': [{'name': '',
     'square': 167.4,
     'country': 'Россия',
     'region': {'id': 63, 'name': 'Москва'},
     'comment': '',
     'type': {'id': 4, 'name': 'Квартира'},
     'own_type': {'id': 22, 'name': 'Совместная собственность'},
     'share': 0.0,
     'relative': None},
    {'name': '',
     'square': 113.9,
     'country': 'Россия',
     'region': None,
     'comment': '',

# ------------------------------------------

# партийный анализ

In [416]:
def get_all_parties():
    pipeline = [ 
        {"$unwind": "$declarations"},
        {"$group": {"_id": "$declarations.main.party", 
                    "declarations": {"$sum": 1},
                   }
        },
    ]
    
    out = db.persons.aggregate(pipeline)
    out = list(out)
    return out

In [418]:
all_parties = get_all_parties()

In [428]:
party_ids = set()
party_names = set()
for entity in all_parties:
#     print(entity)
    if entity['declarations'] > 50 and entity['_id'] is not None:
        party_ids.add(entity['_id']['id'])
        party_names.add(entity['_id']['name'])
print(party_ids, party_names)

{1, 2, 6, 7, 8, 9} {'Патриоты России', 'Единая Россия', 'КПРФ', 'Справедливая Россия', 'ЛДПР', 'Яблоко'}


### какие поля можно вынести в gauge

* средний доход участника
* 